# A modular GCD algorithm: Overview of algorithm

The individual steps are the same as in K. Weber et al. / Journal of Algorithms 54 (2005) 152–167. Given are two integers $a, b \in \mathbb{Z}^*$. This notebook serves as an overview of the algorithm.

First we define some common functions needed by the algorithm.

In [1]:
def mod(b, m):
    """
    Returns the symmetrical modular representation of b mod n
    
    Parameters
    ----------
    b: integer
        An integer.
    m: integer
        The modulus.
        
    Returns
    -------
    y: integer
        Symmetrical modular representation b mod n.
    """
    y = b % m
    if y > m / 2:
        y -= m

    return y

In [2]:
def pi(x):
    """
    Calculates the number of primes less than or equal to the given real number
    
    Parameters
    ----------
    x: real
        A real number.
        
    Returns
    -------
    pi: integer
        Number of primes.
    """
    P = Primes()
    i = 0
    while P.unrank(i) <= x:
        i += 1

    return i

In [3]:
def getPrimes(n):
    primes = set()
    P = Primes()
    i = max(n, 9)
    while len(primes) < n + 2:
        i += 1
        primes.add(P.unrank(i))

    return primes

In [4]:
def findPrime(P, U, V):
    minimum = None
    cur_p = None
    for p in P:
        m = abs(mod(U / V, p))
        if minimum == None or m < minimum:
            minimum = m
            cur_p = p
    
    return cur_p

In [5]:
def getPrimes_mod(number, bound):
    primes = set()
    P = Primes()
    i = 0
    while P.unrank(i) < bound:
        i += 1

    while len(primes) < number:
        assert(i > 0)
        i -= 1
        primes.add(P.unrank(i))

    return primes

In [6]:
def findPrime_mod(P, u, v):
    minimum = None
    current_p = None
    for p in P:
        if v[p] != 0:
            m = abs(mod(u[p] / v[p], p))
            if minimum == None or m < minimum:
                minimum = m
                current_p = p

    return current_p    

Now we implement the algorithm.

In [7]:
def gcd_int(a, b):
    if a >= b:
        U, V = a, b
    else:
        U, V = b, a

    n = floor(log(U, 2)) + 1
    Q = getPrimes(n)
    while V != 0:
        P = set()
        for q in Q:
            if mod(V, q) != 0:
                P.add(q)

        p = findPrime(P, U, V)
        b = mod(U / V, p)
        Q.remove(p)
        U, V = V, (U - b * V) / p
    
    return abs(U)

In [8]:
def gcd_int_mod(a, b):
    if a >= b:
        U, V = a, b
    else:
        U, V = b, a

    n = floor(log(U, 2)) + 1
    w = 1
    while pi(2^w) - pi(2^(w-1)) < max(ceil(2^(w/2) + n), 9):
        w += 1
    
    Q = getPrimes_mod(ceil(2^(w/2) + n), 2^w)
    u, v = {}, {}
    for q in Q:
        u[q], v[q] = mod(U, q), mod(V, q)

    while any(x != 0 for x in v.values()):
        P = set()
        for q in Q:
            if mod(V, q) != 0:
                P.add(q)

        p = findPrime_mod(P, u, v)
        b = mod(u[p] / v[p], p)
        Q.remove(p)
        del u[p], v[p]
        for q in Q:
            u[q], v[q] = v[q], mod((u[q] - b * v[q]) / p, q)    

    G = 0
    while any(x != 0 for x in u.values()):
        p = Q.pop()
        G = u[p] + p * G
        for q in Q:
            u[q] = mod((u[q] - u[p]) / p, q)

        del u[p]

    return abs(G)

Next we test our algorithm on simple inputs.

In [9]:
gcd_int(7 * 5 * 7 * 7 * 3, 7 * 5 * 11 * 5)

35

In [10]:
gcd_int_mod(7 * 5 * 7 * 7 * 3, 7 * 5 * 11 * 5)

35

Finally, we do some performance tests.

In [11]:
primes = set()
P = Primes()
gcd = 1
a = 1
b = 1
for i in range(10):
    gcd *= P.unrank(3 * i)
    a *= P.unrank(3 * i) * P.unrank(3 * i + 2)
    b *= P.unrank(3 * i) * P.unrank(3 * i + 1)

a, b, gcd

(9880780050186010890025694604830,
 2635882834780815441564690163986,
 823933359964162)

In [12]:
timeit('gcd_int(a, b)')

25 loops, best of 3: 7.41 ms per loop


In [13]:
timeit('gcd_int_mod(a, b)')

25 loops, best of 3: 23 ms per loop


In [14]:
primes = set()
P = Primes()
gcd = 1
a = 1
b = 1
for i in range(20):
    gcd *= P.unrank(3 * i)
    a *= P.unrank(3 * i) * P.unrank(3 * i + 2)
    b *= P.unrank(3 * i) * P.unrank(3 * i + 1)

a, b, gcd

(82626625558625627730759447900079022979274827931105265526486455811982536517730,
 16925012608976683449324301595715196440813756060848718135204360679607848186842,
 56737341167208236095244871079519396574)

In [15]:
timeit('gcd_int(a, b)')

5 loops, best of 3: 38.4 ms per loop


In [16]:
timeit('gcd_int_mod(a, b)')

5 loops, best of 3: 80.1 ms per loop


In [17]:
primes = set()
P = Primes()
gcd = 1
a = 1
b = 1
for i in range(30):
    gcd *= P.unrank(3 * i)
    a *= P.unrank(3 * i) * P.unrank(3 * i + 2)
    b *= P.unrank(3 * i) * P.unrank(3 * i + 1)

a, b, gcd

(275152091347708314322123019694904565349090602738899342945202685777899688617140539448385575422596343005949322166003990839148045570,
 46685444725578461804432420389858456875198277594387276894896823729034502064585888014615051084164240555218197063899696338464942162,
 2746257476362523256304808229874501818962789070919585094846865214)

In [18]:
timeit('gcd_int(a, b)')

5 loops, best of 3: 102 ms per loop


In [19]:
timeit('gcd_int_mod(a, b)')

5 loops, best of 3: 177 ms per loop


In [20]:
primes = set()
P = Primes()
gcd = 1
a = 1
b = 1
for i in range(40):
    gcd *= P.unrank(3 * i)
    a *= P.unrank(3 * i) * P.unrank(3 * i + 2)
    b *= P.unrank(3 * i) * P.unrank(3 * i + 1)

a, b, gcd

(3254705835588826997528311319289447239132046896188630394909061658486827365176675594155769214495835743555790944020564693271158958436647641762316283072333105646063501197433729154730773570,
 506923457521246805146359304836956000135243260444180868876073003820256635749451520905220680425309568747329037025468382599270606788979334720220371914032114853474025531611110100336368674,
 8444225397680941067388873318475548909073865271553777534715768151857719836648976324934795518)

In [21]:
timeit('gcd_int(a, b)')

5 loops, best of 3: 204 ms per loop


In [22]:
timeit('gcd_int_mod(a, b)')

5 loops, best of 3: 309 ms per loop


In [23]:
primes = set()
P = Primes()
gcd = 1
a = 1
b = 1
for i in range(50):
    gcd *= P.unrank(3 * i)
    a *= P.unrank(3 * i) * P.unrank(3 * i + 2)
    b *= P.unrank(3 * i) * P.unrank(3 * i + 1)

a, b, gcd

(15540418655783810254423083536796084817197907761730532454791887953358102298034750820114452881224195852385671811698327552913955704962733620200026803774138407575803855013371835262857735909943329591570095099011597054710900523571563968508411178010,
 2215456179871689897461058391695909798308856060579319604339917984386864912630802955495726482297772385581494949353103043986673256033640752796565321657571719402607268852642990196562321575641196913204152035761545013763182008235785025077004689426,
 531906500583557763724707245897905700136086668109311653300767823625429556271590635349581327884550536509787804860213404582)

In [24]:
timeit('gcd_int(a, b)')

5 loops, best of 3: 361 ms per loop


In [25]:
timeit('gcd_int_mod(a, b)')

5 loops, best of 3: 504 ms per loop


In [26]:
primes = set()
P = Primes()
gcd = 1
a = 1
b = 1
for i in range(60):
    gcd *= P.unrank(3 * i)
    a *= P.unrank(3 * i) * P.unrank(3 * i + 2)
    b *= P.unrank(3 * i) * P.unrank(3 * i + 1)

a, b, gcd

(9338456820269818842067617922418465952890590659695436207814129061358299770200340663070553328486393390223249565095962183949737527034189286466618607833768964435472549784475725896743677614279476477276959861494285160216591650746257727144117029451190609825080203686696442053716366727654941894973190009649310,
 1267470851103033146610756067998309940638524988039123516826241805758172060294095018903909984371775827592333056147472976260994026744538655258919960070571117502863210126032694775066747713631499672400830647389238352067592170317939574658731015713054696659143514706931355715650002446477691413081932258868126,
 386270443028608913044690341203130801842385537593800578577960530599852876392121211061482915606221448507167228116709246112625041238986978068053044026602)

In [27]:
timeit('gcd_int(a, b)')

5 loops, best of 3: 582 ms per loop


In [28]:
timeit('gcd_int_mod(a, b)')

5 loops, best of 3: 727 ms per loop


In [29]:
primes = set()
P = Primes()
gcd = 1
a = 1
b = 1
for i in range(70):
    gcd *= P.unrank(3 * i)
    a *= P.unrank(3 * i) * P.unrank(3 * i + 2)
    b *= P.unrank(3 * i) * P.unrank(3 * i + 1)

a, b, gcd

(281116367509022538629449085591914836708398155939839839980431519633305350842374425791219601342376192716695230804728637013181331257147793225510955663579209969253920185185477220993216865592606422769418921182837042802617413451188015791209625711886365090422621863463525861503383551274762531578640136750136924429149586795915390026730296662334273173236090458147447920310,
 36152905671277835716683246937470509506450554196252347334312411591809176203704289639219205230030788922852124807073688927207619141997399578826858810290471682933257573923773599010847774892903761224483990322460188723759041403749965110427249652470374744572138009715815661590099017853869442966433686601541639047253943036760569124606024446961390155156434372928958255206,
 2001955881941388704535139635672257724682729101255573726623652488992082147724661707752203957936278931169636759116042860395358308175905696717852112214430161354017635574807614455513882)

In [30]:
timeit('gcd_int(a, b)')

5 loops, best of 3: 899 ms per loop


In [31]:
timeit('gcd_int_mod(a, b)')

5 loops, best of 3: 1.06 s per loop


In [32]:
primes = set()
P = Primes()
gcd = 1
a = 1
b = 1
for i in range(80):
    gcd *= P.unrank(3 * i)
    a *= P.unrank(3 * i) * P.unrank(3 * i + 2)
    b *= P.unrank(3 * i) * P.unrank(3 * i + 1)

a, b, gcd

(267429445693725191891038788421317000674900814765842175027210718471424884363413092809573371257366443817047032449704963116975167182166110715591248534249769150028715307494644312612561402067260336426989285209131463537514119283645416921148021162330308614514772055930655080464356159291962350580218100519409530877886365001953281955529581816500371779062684186844841256550489114125770779709127868741508488684102993678737503679355718170,
 32884096074020919112566662246082423642933950496456730241531527555810506356314389978893843463378281971523036351842651961070222729388329272983982516094911036408555091073777385881699355960235404583298910699417318780299001873973471626515226375981035718427641556753484395849098682553713581554507477185209540398269666398511598085267534181344873617226126075414479484097060214941008201640999223337690130746368844138486239559150641434,
 5819417690408437853024833101270574716753986259135979159447782308262651211306859356773293060482179913850262056472992485173268331008139501346182

In [33]:
timeit('gcd_int(a, b)')

5 loops, best of 3: 1.27 s per loop


In [34]:
timeit('gcd_int_mod(a, b)')

5 loops, best of 3: 1.46 s per loop


In [35]:
primes = set()
P = Primes()
gcd = 1
a = 1
b = 1
for i in range(90):
    gcd *= P.unrank(3 * i)
    a *= P.unrank(3 * i) * P.unrank(3 * i + 2)
    b *= P.unrank(3 * i) * P.unrank(3 * i + 1)

a, b, gcd

(4318345938067940535048554334343783641665081491352375314566686674699896124016514148984281438653452941503614983920197075086439670021048588476780542297925223295904048189591446045792627337537365459913488279422323501136916583717774586519190374432099398434672696459510996012301813125315127850212841048995014575573604808769379562197988187822146584904050062541123075416571233577093516998758344486251777708717081625514145512368229399071286878147433552854776378387542099869048001449793406858742923390,
 502598474601426543821172867113977633350992954812523102761203324359881119424102977768202183297478481006424085897247016019039082427201669852886376881913777134557436147237812567677537221716065088589576217125267179242890130417154764828203146463781355355382364112071182396626606747079708378235429882655741678749427796930297793281410542460630064347913161495295407594807283156158327017151388786597952807263534726875038172499921118258783086846069522121540334784667291951392034240771551081407541738,
 70613945778084

In [36]:
timeit('gcd_int(a, b)')

5 loops, best of 3: 1.66 s per loop


In [37]:
timeit('gcd_int_mod(a, b)')

5 loops, best of 3: 1.72 s per loop


In [38]:
primes = set()
P = Primes()
gcd = 1
a = 1
b = 1
for i in range(100):
    gcd *= P.unrank(3 * i)
    a *= P.unrank(3 * i) * P.unrank(3 * i + 2)
    b *= P.unrank(3 * i) * P.unrank(3 * i + 1)

a, b, gcd

(1038880448762075294101899484771645187553714410435580341310798228046553771491604269564254500936828071081524568850844559184943472857035101129153194121592883394767472732298512999555155226781675823402235320813604503233259605459553533998301548853734077971863923488822373986867494462661655088518233455592359716213521278853679153173931552856872002165770044422374583464645650593147300763787323107857098838320659913596541623373352450350101268941511518224689138847606655448706496353978712497677083548887363205386512289958282478052900742342801038762150912289743202810,
 11571027579229306575202896805157580789136609165146655870060872574107452121281978945804832073249900946049087439672947794403156144114317795397585189132531635291229605171982245188040073926129339828197474996920681454692202815198294802486484842664556533871055243808497963634233471607303125087946160985093546458427608207207370590908638636806947378869499130342575444345472821677394137017632633296191555633172241156112131351413772824812453467485518

In [39]:
timeit('gcd_int(a, b)')

5 loops, best of 3: 2.18 s per loop


In [40]:
timeit('gcd_int_mod(a, b)')

5 loops, best of 3: 2.18 s per loop


In [41]:
primes = set()
P = Primes()
gcd = 1
a = 1
b = 1
for i in range(110):
    gcd *= P.unrank(3 * i)
    a *= P.unrank(3 * i) * P.unrank(3 * i + 2)
    b *= P.unrank(3 * i) * P.unrank(3 * i + 1)

a, b, gcd

(2651678953686198784542815713596132257594670268979830338967821310405661407908459493437311983116748165733130262679031561431291994863136050882246851801065289223852430261495164922014904762084717663632472989212711319949949162871885846018549133750261468263728666559115025196054235776802205722212328981947523121570745565974962488427990592879331980917939759343904002608527747912103214462322833320532491647593697663577077871039572460165069876161792695410382113458409543785675470824020011015307407636991616629672499131586672151641592121382107587549591653419958314538890574343143559785474767595915553874190481360488363153553711596490,
 28595552295200245734585343966160320686007498233322639998592635630070059664741751156782297400391402034938109910546302029886041780230676930835238655377298963312833976913007502722845777579986356888322078086844751960362283812361280360426323485195893912696304958030509317551551649661172298161994259671291362887650053985458571089342505792159780845307115943602736436497698764769118

In [42]:
timeit('gcd_int(a, b)')

5 loops, best of 3: 3.03 s per loop


In [43]:
timeit('gcd_int_mod(a, b)')

5 loops, best of 3: 2.82 s per loop


In [44]:
primes = set()
P = Primes()
gcd = 1
a = 1
b = 1
for i in range(120):
    gcd *= P.unrank(3 * i)
    a *= P.unrank(3 * i) * P.unrank(3 * i + 2)
    b *= P.unrank(3 * i) * P.unrank(3 * i + 1)

a, b, gcd

(56991192054608294346588959436400470523932215523218350318172374025393324087870998261899585458885749899187891167710604232190960335632229423927337342729327245734933495200275024568058942228691335991918013378198317378256316041445330893800521662972812333455378993526206623463757873403629772946975314747995514268558074837333775280211643804969548046590398545123000466347707746406663893512351531701506205540774654108311594124295530199192823282665187009954691494624479544550539265110317924124165834055991752056905471479586289121412615890067170756789864290259992497126579310411331196609249345350586887801774884771859597937157498602893626326101895494804932732738276168561564074242924390590256055030530,
 5922580371257739588793678693377999025479217807815790444490821054407021754594118080054903511795351717082412529465357668445343671339253264348267306970514841123617842994284691298080899515281620681421865245970838923384164815884806057265335128894781873743075274086246035213823655766325117008398735355191510756683

In [45]:
timeit('gcd_int(a, b)')

5 loops, best of 3: 3.64 s per loop


In [46]:
timeit('gcd_int_mod(a, b)')

5 loops, best of 3: 3.27 s per loop
